# Preparation
Below are the steps from the original code to train and evaluate the model.

## Official code for FriendNet and YOLOv7

In [ ]:
!git clone https://github.com/fanyihua0309/FriendNet.git
%cd FriendNet
!git clone https://github.com/bubbliiiing/yolov7-tiny-pytorch

## Datasets
Download and unzip VOC-FOG dataset. Due to shortage of time, I cannot perform demo on Foggy-Driving dataset, however, the dataset can be found here https://www.kaggle.com/datasets/washingtongold/foggy-driving

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1bLUtwrKwzPwLI3yZBFZYw4BnINpxCfVp/view
!unzip voc-fog(9578+2129).zip

Move data to `FriendNet/data`, the folder structure is as below:

```python
┬─ data
│   ├─ VOC-FOG
│   │   ├─ train
│   │   │   ├─ FogImages
│   │   │   ├─ JPEGImages
│   │   │   └─ Annotations
│   │   └─ test
│   │       ├─ FogImages
│   │       ├─ JPEGImages
│   │       └─ Annotations
│   └─ Foggy-Driving
│       └─ test
│           ├─ FogImages
│           ├─ JPEGImages
│           └─ Annotations
```

Next, parse detection annotations and generate necessary text files for training, namely `train.txt` and `test.txt`. To train yolov7 model, remember to modify `convert_VOC_to_txt.py` to generate suitable annotations

In [ ]:
!python convert_VOC_to_txt.py

# Train YOLOv7

Create a `train_detector.py` file to train yolov7 detector following the `train.py` from yolov7-tiny-pytorch repository \
Specify path for input labels, classes,... then train

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=2 train_detector.py

Results can be found at `logs/`

# Train Dehaze Network
Create a `train_dehaze.py` file. Steps in one epoch would be:
*   get detection guidance from trained yolov7
*   feed guidance and hazy image into network
*   outputs are used to calculate restoration loss
*   calculate final loss

Some details that need modification


*   `get_detection_guidance()` implementation
*   Roll back `covert_VOC_to_txt.py` to get original format

Execute the below script to train dehaze network. Results can be found at `logs_dehazed/`




In [ ]:
!python train_dehaze.py

# Test

Run the below script to generate dehazed images. Change the `--dehaze_model_path` argument as you wish

In [ ]:
!python test_dehaze.py --input_dir=data/VOC-FOG/test/FogImages --output_dir=data/VOC-FOG/test/DehazedImages --dehaze_model_path=logs_dehazed/best_model.pth

Run the below script to generate detection results. Change the `--detech_model_path` argument as you wish


In [ ]:
!python predict_detect.py --input_dir=data/VOC-FOG/test/DehazedImages --output_dir=data/VOC-FOG/test/DetectedImages --detect_model_path=logs/yolov7-tiny_clean_best_epoch_weights.pth

# Evaluation

For fast evaluation, specify the path to hazy images, dehazed images, groundtruth images and labels folders. Then run the below script to get evaluation results including dehazed images, detection results, mAP, PSNR, SSIM.

In [ ]:
!python quick_eval_script.py 

Some results: 
```python
PSNR=29.1050, SSIM=0.9260

37.55% = bicycle AP     ||      score_threhold=0.5 : F1=0.00 ; Recall=0.00% ; Precision=0.00%
56.19% = bus AP         ||      score_threhold=0.5 : F1=0.13 ; Recall=7.19% ; Precision=66.67%
56.99% = car AP         ||      score_threhold=0.5 : F1=0.36 ; Recall=22.69% ; Precision=88.29%
40.67% = motorbike AP   ||      score_threhold=0.5 : F1=0.02 ; Recall=0.76% ; Precision=50.00%
72.68% = person AP      ||      score_threhold=0.5 : F1=0.59 ; Recall=45.09% ; Precision=83.47%
mAP = 52.82%
```

Here is the link to my source code:
https://github.com/hongquant17/FriendNet \
Due to the privacy of the Residency Program, I will set the repository to private. Please contact me for access.